In [97]:
from src.Webscraping import Webscraping_TrailData as TD
from src.Webscraping import Webscraping_TrailReports as TR
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import pymongo
import math
import pandas as pd
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count
import numpy as np 

In [9]:
hike_urls = pd.read_csv('data/olympics_trail_data.csv')

In [81]:
hike_urls

,Coast,Dogs not allowed,Established campsites,Fall foilage,Good for kids,Lakes,Mountain views,Old growth,Ridges/passes,Rivers,...,highest_point,hike_name,lat,long,numReports,number_votes,rating,region,url,which_pass
0,1,0,0,0,0,0,0,0,0,1,...,NaN,Maple Hollow Park,47.2955,-122.7509,1,NaN,NaN,NaN,https://www.wta.org/go-hiking/hikes/maple-holl...,NaN
1,1,0,0,0,1,0,1,0,0,0,...,NaN,Point No Point Park,47.9115,-122.5291,17,NaN,NaN,NaN,https://www.wta.org/go-hiking/hikes/point-no-p...,None
2,0,1,0,0,1,0,0,0,0,0,...,Highest Point: 50 ft.\n,Mary E. Theler Wetlands Nature Preserve,47.4381,-122.8366,21,(11 votes) Log in to rate,3.09 out of 5,Olympic Peninsula -- Kitsap Peninsula,https://www.wta.org/go-hiking/hikes/mary-e-the...,NaN
3,1,1,0,0,1,0,1,1,0,1,...,Highest Point: 360 ft.\n,Guillemot Cove,47.6160,-122.9085,53,(14 votes) Log in to rate,4.07 out of 5,Olympic Peninsula -- Kitsap Peninsula,https://www.wta.org/go-hiking/hikes/guillemot-...,None
4,1,0,0,0,1,0,1,1,0,0,...,Highest Point: 25 ft.\n,Kopachuck State Park Trail,47.3085,-122.6807,9,(2 votes) Log in to rate,3.50 out of 5,Olympic Peninsula -- Kitsap Peninsula,https://www.wta.org/go-hiking/hikes/kopachuck-...,Discover Pass
5,1,1,0,0,1,0,1,0,0,0,...,Highest Point: 130 ft.\n,Dungeness Spit,48.1413,-123.1899,131,(21 votes) Log in to rate,3.48 out of 5,Olympic Peninsula -- Northern Coast,https://www.wta.org/go-hiking/hikes/dungeness-...,Refuge Entrance Pass
6,0,0,1,0,0,0,1,1,0,1,...,Highest Point: 1850 ft.\n,Big Creek,47.4933,-123.2110,189,(26 votes) Log in to rate,3.92 out of 5,Olympic Peninsula -- Hood Canal,https://www.wta.org/go-hiking/hikes/big-creek,Northwest Forest Pass
7,0,0,0,0,0,0,1,0,0,0,...,Highest Point: 2804 ft.\n,Mount Walker,47.7756,-122.9137,235,(20 votes) Log in to rate,3.20 out of 5,Olympic Peninsula -- Hood Canal,https://www.wta.org/go-hiking/hikes/mount-walker,None
8,1,0,0,0,1,0,0,0,0,0,...,Highest Point: 200 ft.\n,Burfoot Park,47.1325,-122.9017,7,(5 votes) Log in to rate,3.00 out of 5,Olympic Peninsula -- Olympia,https://www.wta.org/go-hiking/hikes/burfoot-park,NaN
9,1,1,1,0,0,0,0,0,0,0,...,Highest Point: 350 ft.\n,Cape Alava Loop (Ozette Triangle),48.1536,-124.6679,242,(43 votes) Log in to rate,4.14 out of 5,Olympic Peninsula -- Pacific Coast,https://www.wta.org/go-hiking/hikes/lake-ozette,National Park Pass


In [80]:
hike_urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 26 columns):
Coast                    360 non-null int64
Dogs not allowed         360 non-null int64
Established campsites    360 non-null int64
Fall foilage             360 non-null int64
Good for kids            360 non-null int64
Lakes                    360 non-null int64
Mountain views           360 non-null int64
Old growth               360 non-null int64
Ridges/passes            360 non-null int64
Rivers                   360 non-null int64
Summits                  360 non-null int64
Waterfalls               360 non-null int64
Wildflowers/Meadows      360 non-null int64
Wildlife                 360 non-null int64
distance                 276 non-null object
elevation_gain           276 non-null object
highest_point            276 non-null object
hike_name                360 non-null object
lat                      360 non-null float64
long                     360 non-null float64
numRepo

In [111]:
hike_urls['hike_name']

0                               Maple Hollow Park
1                             Point No Point Park
2         Mary E. Theler Wetlands Nature Preserve
3                                  Guillemot Cove
4                      Kopachuck State Park Trail
5                                  Dungeness Spit
6                                       Big Creek
7                                    Mount Walker
8                                    Burfoot Park
9               Cape Alava Loop (Ozette Triangle)
10            Hoh River Trail to Five Mile Island
11                                Upper Lena Lake
12                          Upper Dungeness River
13                          Spruce Railroad Trail
14                          Lower Gray Wolf River
15                                 Mount Townsend
16                                      Lena Lake
17          Shi Shi Beach and Point of the Arches
18                                  Mount Ellinor
19               Marmot Pass - Upper Big Quilcene


In [109]:
one = hike_urls['highest_point'][2]
int(re.findall('\d+', one )[0])


50

In [98]:
def total_dst(string):
    """Extracts mileage from string and calculates total mileage depending on if
    description says one-way or roundtrip."""

    if str(string) == 'nan':
        return np.nan
    else:
        miles = []
        try:
            for s in string.split():
                miles.append(float(s))
        except:
            miles.append(string)
        if 'roundtrip' in string:
            return miles[0]
        elif 'one-way' in string:
            return miles[0]*2
        else:
            return miles[0]

In [112]:
def num_votes(string):
    '''Extracts number of votes'''
    lst = string.strip('(').split()
    if 'NR' in lst:
        return 'NR'
    else:
        return int(lst[0])

In [115]:
num_votes(hike_urls['number_votes'][0])

AttributeError: 'float' object has no attribute 'strip'

In [113]:
hike_urls['number_votes'].apply(lambda x: num_votes(x))

AttributeError: 'float' object has no attribute 'strip'

In [104]:
hike_urls['total_distance'] = hike_urls['distance'].apply(lambda x: total_dst(x))


TypeError: argument of type 'float' is not iterable

In [52]:
hike_urls[hike_urls['highest_point'].isna() == False]['highest_point'].apply(lambda x: int(re.findall('\d+', x)[0]))

2        50
3       360
4        25
5       130
6      1850
7      2804
8       200
9       350
10      800
11     4600
12     3100
13      700
14     1500
15     6260
16     2000
17      200
18     5944
19     6000
20     1639
21     2500
22      350
26     1100
28     4300
32       50
33      360
34       25
35      130
36     1850
37     2804
38      200
       ... 
318    5944
319    6000
320    1639
321    2500
322     350
326    1100
328    4300
332      50
333     360
334      25
335     130
336    1850
337    2804
338     200
339     350
340     800
341    4600
342    3100
343     700
344    1500
345    6260
346    2000
347     200
348    5944
349    6000
350    1639
351    2500
352     350
356    1100
358    4300
Name: highest_point, Length: 276, dtype: int64

In [45]:
spot = hike_urls['highest_point'][2]
spot

'Highest Point: 50 ft.\n'

AttributeError: 'list' object has no attribute 'join'

In [53]:
import re
int(re.findall('\d+', spot )[0])

50

In [54]:
    mc = pymongo.MongoClient()
    db = mc['wta']
    trail_reports = db['trail_reports']
    raw_html = db['html']
    trail_page_raw_html = db['trail_html']

In [57]:
df = pd.DataFrame(list(trail_reports.find()))

In [78]:
df

,Creator,Date,Report,Trail,Trail_condtions,Votes,_id,Date_type
0,JoeHendricks,"Jun 04, 2013","The recently re-opened DNR park, Maple Hollow,...",Maple Hollow Park,None,None,5af219fcacf3d67636dc352d,2013-06-04
1,ellieoutside,"Feb 26, 2018",I just wasn't feeling like putting on all the ...,Point No Point Park,None,None,5af219feacf3d67636dc3530,2018-02-26
2,ufda94,"Jan 30, 2018",This was our 1st Experience hiking in Washingt...,Point No Point Park,None,1,5af219feacf3d67636dc3531,2018-01-30
3,Larnruoff,"Jan 04, 2018",A short jaunt through a bit of woods down the ...,Point No Point Park,None,None,5af219feacf3d67636dc3532,2018-01-04
4,ejain,"Dec 23, 2017",Started from the smaller parking lot off Loki ...,Point No Point Park,None,1,5af219feacf3d67636dc3533,2017-12-23
5,Basgla224,"Dec 03, 2017","Nice variety, walking along the water, by wetl...",Point No Point Park,None,4,5af219feacf3d67636dc3534,2017-12-03
6,Kiloa,"Nov 06, 2017",Nice hike on a cold and crisp but sunny day. I...,Point No Point Park,None,2,5af219ffacf3d67636dc3536,2017-11-06
7,Loki from Tacoma,"Oct 01, 2017",We came up from Tacoma for this short hike tod...,Point No Point Park,None,None,5af219ffacf3d67636dc3537,2017-10-01
8,MindyLea,"Jul 12, 2017",I don't know about hiking because we parked in...,Point No Point Park,None,2,5af219ffacf3d67636dc3538,2017-07-12
9,Drew B.,"Apr 23, 2017",If you happen upon North Kitsap or have an aff...,Point No Point Park,None,2,5af219ffacf3d67636dc3539,2017-04-23


In [77]:
df['Date_type'] = pd.to_datetime(df['Date'])

In [ ]:
df['Creator'] = df['Creator'].apply(lambda x: x.strip('\n'))